In [15]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPool2D, Flatten, Dense, Dropout

import pickle
import time


path_to_train_images = './Face_NoFace'

train_datagen = ImageDataGenerator(
    shear_range=0.1, zoom_range=0.1, horizontal_flip=True)
test_datagen = ImageDataGenerator()

x_train = train_datagen.flow_from_directory(
    path_to_train_images, target_size=(64, 64), batch_size=32, class_mode='categorical')
x_test = test_datagen.flow_from_directory(path_to_train_images, 
    target_size=(64, 64), batch_size=32, class_mode='categorical')

Y_train = x_train.class_indices


res_map = {}
for face_val, face_name in zip(Y_train.values(), Y_train.keys()):
    res_map[face_val] = face_name

with open("ResultsMap.pkl", 'wb') as file:
    pickle.dump(res_map, file)

output_neurons = len(res_map)

cnn_model = Sequential([
    Convolution2D(32, kernel_size=(5, 5), strides=(1, 1),
                  input_shape=(64, 64, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Convolution2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(output_neurons, activation='softmax')
])

cnn_model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=["accuracy"])


start_train_time = time.time()

cnn_model.fit(x_train, epochs=20, validation_data=x_test)

end_train_time = time.time()
print('Time taken: ', round((end_train_time - start_train_time)), 'seconds')

cnn_model.save("face_detect.keras")

Found 2654 images belonging to 2 classes.
Found 2654 images belonging to 2 classes.
Epoch 1/20
83/83 [==============================] - 87s 1s/step - loss: 7.8619 - accuracy: 0.7065 - val_loss: 0.4760 - val_accuracy: 0.8007
Epoch 2/20
83/83 [==============================] - 85s 1s/step - loss: 0.4471 - accuracy: 0.8078 - val_loss: 0.3987 - val_accuracy: 0.8335
Epoch 3/20
83/83 [==============================] - 84s 1s/step - loss: 0.4046 - accuracy: 0.8338 - val_loss: 0.3572 - val_accuracy: 0.8417
Epoch 4/20
83/83 [==============================] - 83s 1s/step - loss: 0.3698 - accuracy: 0.8444 - val_loss: 0.3249 - val_accuracy: 0.8655
Epoch 5/20
83/83 [==============================] - 83s 1s/step - loss: 0.3658 - accuracy: 0.8444 - val_loss: 0.2846 - val_accuracy: 0.8904
Epoch 6/20
83/83 [==============================] - 84s 1s/step - loss: 0.2728 - accuracy: 0.8960 - val_loss: 0.1766 - val_accuracy: 0.9378
Epoch 7/20
83/83 [==============================] - 84s 1s/step - loss: 0.19

/home/phucle03/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [5]:
from keras.preprocessing import image
import keras
import numpy as np
import pickle
import os


cnn_model = keras.models.load_model("face_detect.keras")
res_map_file = "ResultsMap.pkl"
res_map = {}
with open(res_map_file, 'rb') as file:
    unpickler = pickle.Unpickler(file)
    res_map = unpickler.load()
    
imgfile = "/home/phucle03/Phuc/Projects/DoAn/Smart-Store/Backend/temp/imgs/0788b05d-2e5d-49b1-afb5-f877eab6d953.jpg"
imgdir = "./Face Images/TestUnknown/"

for img in ["phuc.jpg", "phuong.jpg", "obama.jpg", "biden.jpg", "qhuy.jpg", "trump.jpg", "girl.png", "cat.png", "dog.png", "horse.jpg","flower.png"]:
    imgfile = os.path.join(imgdir, img)

    test_image = image.load_img(imgfile, target_size=(64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = cnn_model.predict(test_image, verbose=0)

    print('Prediction is: ', img, res_map[np.argmax(result)])

Prediction is:  phuc.jpg 1
Prediction is:  phuong.jpg 1
Prediction is:  obama.jpg 1
Prediction is:  biden.jpg 1
Prediction is:  qhuy.jpg 1
Prediction is:  trump.jpg 1
Prediction is:  girl.png 1
Prediction is:  cat.png 0
Prediction is:  dog.png 0
Prediction is:  horse.jpg 0
Prediction is:  flower.png 0
